In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from catboost import CatBoostClassifier
import pandas as pd
import json
from pandas import json_normalize
from tqdm import tqdm_notebook as tqdm
import numpy as np
import ast
import time
import datetime

import sys
import os
sys.path.append(os.path.join(sys.path[0], '../../core/'))

from train_utils import calculate_roi, get_winner_favorite, combine_df, parse_odds

pd.set_option('display.precision',6)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [3]:
def load_fighters():
    fighters_df = pd.read_csv("data_models/fighters_df.csv", index_col=0)

    fighters_df["dateOfBirth"] = pd.to_datetime(fighters_df["dateOfBirth"])
    fighters_cols = [
        # "id",
        "name",
        "weight",
        "height",
        "armSpan",
        "legSwing",
        "weightCategory.id",
        "weightCategory.name",
        "dateOfBirth",
        "country",
        "city",
        "timezone",
    ]
    fighters_df = fighters_df[fighters_cols]
    # fighters_df.set_index("id", inplace=True)
    f_name_dict = fighters_df['name'].to_dict()

    return fighters_df, f_name_dict

In [4]:
static_cols = ['country', 'city', 'armSpan', 'height', 'legSwing', 'timezone', 'weight']
f1_static_cols = ['f1_' + col for col in static_cols]
f2_static_cols = ['f2_' + col for col in static_cols]

cat_cols = ['country', 'city', 'timezone']
f1_cat_cols = ['f1_' + col for col in cat_cols]
f2_cat_cols = ['f2_' + col for col in cat_cols]

fighters_df, f_name_dict = load_fighters()
with open('data_models/generated_features_11.05.2021.txt', 'r') as outfile:
    generated_features = json.load(outfile)
num_cols = [i[3:] for i in generated_features['fighter1_stats']][:-8]
num_cols

['cumsum_duration',
 'cumsum_winner',
 'cumsum_hitsTotal',
 'cumsum_hitsSuccessful',
 'cumsum_takedownTotal',
 'cumsum_takedownSuccessful',
 'cumsum_submissionAttempts',
 'cumsum_takeovers',
 'cumsum_accentedHitsTotal',
 'cumsum_accentedHitsSuccessful',
 'cumsum_knockdowns',
 'cumsum_protectionPassage',
 'cumsum_hitsHeadTotal',
 'cumsum_hitsHeadSuccessful',
 'cumsum_hitsBodyTotal',
 'cumsum_hitsBodySuccessful',
 'cumsum_hitsLegsTotal',
 'cumsum_hitsLegsSuccessful',
 'cumsum_accentedHitsPositionDistanceTotal',
 'cumsum_accentedHitsPositionDistanceSuccessful',
 'cumsum_accentedHitsPositionClinchTotal',
 'cumsum_accentedHitsPositionClinchSuccessful',
 'cumsum_accentedHitsPositionParterTotal',
 'cumsum_accentedHitsPositionParterSuccessful',
 'cumsum_def_hitsTotal',
 'cumsum_def_hitsSuccessful',
 'cumsum_def_takedownTotal',
 'cumsum_def_takedownSuccessful',
 'cumsum_def_submissionAttempts',
 'cumsum_def_takeovers',
 'cumsum_def_accentedHitsTotal',
 'cumsum_def_accentedHitsSuccessful',
 'cum

In [5]:
f_stats_events_cumulative = pd.read_csv('data_models/PROD_f_stats_events_cumulative_prod_19.05.2021.csv', index_col=0)
f_stats_events_cumulative['eventDate.date'] = pd.to_datetime(f_stats_events_cumulative['eventDate.date'])
f_stats_events_cumulative

,cumsum_duration,cumsum_winner,cumsum_hitsTotal,cumsum_hitsSuccessful,cumsum_takedownTotal,cumsum_takedownSuccessful,cumsum_submissionAttempts,cumsum_takeovers,cumsum_accentedHitsTotal,cumsum_accentedHitsSuccessful,cumsum_knockdowns,cumsum_protectionPassage,cumsum_hitsHeadTotal,cumsum_hitsHeadSuccessful,cumsum_hitsBodyTotal,cumsum_hitsBodySuccessful,cumsum_hitsLegsTotal,cumsum_hitsLegsSuccessful,cumsum_accentedHitsPositionDistanceTotal,cumsum_accentedHitsPositionDistanceSuccessful,cumsum_accentedHitsPositionClinchTotal,cumsum_accentedHitsPositionClinchSuccessful,cumsum_accentedHitsPositionParterTotal,cumsum_accentedHitsPositionParterSuccessful,cumsum_def_hitsTotal,cumsum_def_hitsSuccessful,cumsum_def_takedownTotal,cumsum_def_takedownSuccessful,cumsum_def_submissionAttempts,cumsum_def_takeovers,cumsum_def_accentedHitsTotal,cumsum_def_accentedHitsSuccessful,cumsum_def_knockdowns,cumsum_def_protectionPassage,cumsum_def_hitsHeadTotal,cumsum_def_hitsHeadSuccessful,cumsum_def_hitsBodyTotal,cumsum_def_hitsBodySuccessful,cumsum_def_hitsLegsTotal,cumsum_def_hitsLegsSuccessful,cumsum_def_accentedHitsPositionDistanceTotal,cumsum_def_accentedHitsPositionDistanceSuccessful,cumsum_def_accentedHitsPositionClinchTotal,cumsum_def_accentedHitsPositionClinchSuccessful,cumsum_def_accentedHitsPositionParterTotal,cumsum_def_accentedHitsPositionParterSuccessful,cumsum_winMethods_[DEC],cumsum_winMethods_[DQ],cumsum_winMethods_[KO],cumsum_winMethods_[SUB],current_duration,winner,current_hitsTotal,current_hitsSuccessful,current_takedownTotal,current_takedownSuccessful,current_submissionAttempts,current_takeovers,current_accentedHitsTotal,current_accentedHitsSuccessful,current_knockdowns,current_protectionPassage,current_hitsHeadTotal,current_hitsHeadSuccessful,current_hitsBodyTotal,current_hitsBodySuccessful,current_hitsLegsTotal,current_hitsLegsSuccessful,current_accentedHitsPositionDistanceTotal,current_accentedHitsPositionDistanceSuccessful,current_accentedHitsPositionClinchTotal,current_accentedHitsPositionClinchSuccessful,current_accentedHitsPositionParterTotal,current_accentedHitsPositionParterSuccessful,current_def_hitsTotal,current_def_hitsSuccessful,current_def_takedownTotal,current_def_takedownSuccessful,current_def_submissionAttempts,current_def_takeovers,current_def_accentedHitsTotal,current_def_accentedHitsSuccessful,current_def_knockdowns,current_def_protectionPassage,current_def_hitsHeadTotal,current_def_hitsHeadSuccessful,current_def_hitsBodyTotal,current_def_hitsBodySuccessful,current_def_hitsLegsTotal,current_def_hitsLegsSuccessful,current_def_accentedHitsPositionDistanceTotal,current_def_accentedHitsPositionDistanceSuccessful,current_def_accentedHitsPositionClinchTotal,current_def_accentedHitsPositionClinchSuccessful,current_def_accentedHitsPositionParterTotal,current_def_accentedHitsPositionParterSuccessful,current_winMethods_[DEC],current_winMethods_[DQ],current_winMethods_[KO],current_winMethods_[SUB],age,eventDate.date,fighterId,fighter_nbr,odds,winnerId,count_of_fights,hits_accuracy,takedown_accuracy,accentedHits_accuracy,hitsHead_accuracy,hitsBody_accuracy,hitsLegs_accuracy,accentedHitsPositionDistance_accuracy,accentedHitsPositionClinch_accuracy,accentedHitsPositionParter_accuracy,def_hits_accuracy,def_takedown_accuracy,def_accentedHits_accuracy,def_hitsHead_accuracy,def_hitsBody_accuracy,def_hitsLegs_accuracy,def_accentedHitsPositionDistance_accuracy,def_accentedHitsPositionClinch_accuracy,def_accentedHitsPositionParter_accuracy,DEC_percent,DQ_percent,KO_percent,SUB_percent,hits_PM,takedown_PM,accentedHits_PM,hitsHead_PM,hitsBody_PM,hitsLegs_PM,accentedHitsPositionDistance_PM,accentedHitsPositionClinch_PM,accentedHitsPositionParter_PM,def_hits_PM,def_takedown_PM,def_accentedHits_PM,def_hitsHead_PM,def_hitsBody_PM,def_hitsLegs_PM,def_accentedHitsPositionDistance_PM,def_accentedHitsPositionClinch_PM,def_accentedHitsPositionParter_PM,knockdowns_PM,def_knockdowns_PM,protectionPassage_PM,def_protectionPassage_PM,win_streak,loose_s

In [26]:
clf = CatBoostClassifier()
clf.load_model('data_models/catboost_v2_1_19.05.2021.cat')
f1_id, f2_id = 141, 453
event_date, time_zone = datetime.date(2021, 1, 16), 'Europe/Berlin'
f1_odd, f2_odd = 3.37, 1.34
weightCategory_id, event_name = 6, 'UFC Fight Night'
city, country = 'Abu Dhabi', 'United Arab Emirates'
clf.feature_names_


f1_id, f2_id = 3610, 3664
event_date, time_zone = datetime.date(2021, 1, 20), 'Europe/Berlin'
f1_odd, f2_odd = 3.37, 1.34
weightCategory_id, event_name = 6, 'UFC Fight Night'
city, country = 'Abu Dhabi', 'United Arab Emirates'
clf.feature_names_

['age_difference',
 'armSpan_difference',
 'cumsum_def_hitsTotal_difference',
 'cumsum_hitsSuccessful_difference',
 'cumsum_submissionAttempts_difference',
 'def_accentedHitsPositionDistance_accuracy_difference',
 'def_hitsHead_accuracy_difference',
 'def_hitsLegs_PM_difference',
 'def_hitsLegs_accuracy_difference',
 'cumsum_takeovers_difference',
 'cumsum_takedownTotal_difference',
 'isHomeCity_difference',
 'isHomeCountry_difference',
 'takedown_PM_difference',
 'win_streak_difference']

In [27]:
is_fight_night = 'fight night' in str(event_name).lower()

### Static stats
f1_birthDate = fighters_df.loc[int(f1_id), ['dateOfBirth']]
f1_static_stats = fighters_df.loc[int(f1_id), static_cols].values
f1_age = ((pd.to_datetime(event_date) - pd.to_datetime(f1_birthDate)) / 365).dt.days.values[0]

f2_birthDate = fighters_df.loc[int(f2_id), ['dateOfBirth']]
f2_static_stats = fighters_df.loc[int(f2_id), static_cols].values
f2_age = ((pd.to_datetime(event_date) - pd.to_datetime(f2_birthDate)) / 365).dt.days.values[0]

### Dynamic stats
fighter1_stats = f_stats_events_cumulative[
    (f_stats_events_cumulative['fighterId'] == int(f1_id)) &
    (f_stats_events_cumulative['eventDate.date'] < pd.to_datetime(event_date))]
# fighter1_stats = fighter1_stats.iloc[[-1]].reset_index(drop=True)
fighter1_stats['cumsum_hitsTotal']

Series([], Name: cumsum_hitsTotal, dtype: int64)

In [28]:
fighter1_stats.shape[0]

0

In [30]:
fighter2_stats = f_stats_events_cumulative[
    (f_stats_events_cumulative['fighterId'] == int(f2_id)) &
    (f_stats_events_cumulative['eventDate.date'] < pd.to_datetime(event_date))]
# fighter2_stats = fighter2_stats.iloc[[-1]].reset_index(drop=True)
fighter2_stats


,cumsum_duration,cumsum_winner,cumsum_hitsTotal,cumsum_hitsSuccessful,cumsum_takedownTotal,cumsum_takedownSuccessful,cumsum_submissionAttempts,cumsum_takeovers,cumsum_accentedHitsTotal,cumsum_accentedHitsSuccessful,cumsum_knockdowns,cumsum_protectionPassage,cumsum_hitsHeadTotal,cumsum_hitsHeadSuccessful,cumsum_hitsBodyTotal,cumsum_hitsBodySuccessful,cumsum_hitsLegsTotal,cumsum_hitsLegsSuccessful,cumsum_accentedHitsPositionDistanceTotal,cumsum_accentedHitsPositionDistanceSuccessful,cumsum_accentedHitsPositionClinchTotal,cumsum_accentedHitsPositionClinchSuccessful,cumsum_accentedHitsPositionParterTotal,cumsum_accentedHitsPositionParterSuccessful,cumsum_def_hitsTotal,cumsum_def_hitsSuccessful,cumsum_def_takedownTotal,cumsum_def_takedownSuccessful,cumsum_def_submissionAttempts,cumsum_def_takeovers,cumsum_def_accentedHitsTotal,cumsum_def_accentedHitsSuccessful,cumsum_def_knockdowns,cumsum_def_protectionPassage,cumsum_def_hitsHeadTotal,cumsum_def_hitsHeadSuccessful,cumsum_def_hitsBodyTotal,cumsum_def_hitsBodySuccessful,cumsum_def_hitsLegsTotal,cumsum_def_hitsLegsSuccessful,cumsum_def_accentedHitsPositionDistanceTotal,cumsum_def_accentedHitsPositionDistanceSuccessful,cumsum_def_accentedHitsPositionClinchTotal,cumsum_def_accentedHitsPositionClinchSuccessful,cumsum_def_accentedHitsPositionParterTotal,cumsum_def_accentedHitsPositionParterSuccessful,cumsum_winMethods_[DEC],cumsum_winMethods_[DQ],cumsum_winMethods_[KO],cumsum_winMethods_[SUB],current_duration,winner,current_hitsTotal,current_hitsSuccessful,current_takedownTotal,current_takedownSuccessful,current_submissionAttempts,current_takeovers,current_accentedHitsTotal,current_accentedHitsSuccessful,current_knockdowns,current_protectionPassage,current_hitsHeadTotal,current_hitsHeadSuccessful,current_hitsBodyTotal,current_hitsBodySuccessful,current_hitsLegsTotal,current_hitsLegsSuccessful,current_accentedHitsPositionDistanceTotal,current_accentedHitsPositionDistanceSuccessful,current_accentedHitsPositionClinchTotal,current_accentedHitsPositionClinchSuccessful,current_accentedHitsPositionParterTotal,current_accentedHitsPositionParterSuccessful,current_def_hitsTotal,current_def_hitsSuccessful,current_def_takedownTotal,current_def_takedownSuccessful,current_def_submissionAttempts,current_def_takeovers,current_def_accentedHitsTotal,current_def_accentedHitsSuccessful,current_def_knockdowns,current_def_protectionPassage,current_def_hitsHeadTotal,current_def_hitsHeadSuccessful,current_def_hitsBodyTotal,current_def_hitsBodySuccessful,current_def_hitsLegsTotal,current_def_hitsLegsSuccessful,current_def_accentedHitsPositionDistanceTotal,current_def_accentedHitsPositionDistanceSuccessful,current_def_accentedHitsPositionClinchTotal,current_def_accentedHitsPositionClinchSuccessful,current_def_accentedHitsPositionParterTotal,current_def_accentedHitsPositionParterSuccessful,current_winMethods_[DEC],current_winMethods_[DQ],current_winMethods_[KO],current_winMethods_[SUB],age,eventDate.date,fighterId,fighter_nbr,odds,winnerId,count_of_fights,hits_accuracy,takedown_accuracy,accentedHits_accuracy,hitsHead_accuracy,hitsBody_accuracy,hitsLegs_accuracy,accentedHitsPositionDistance_accuracy,accentedHitsPositionClinch_accuracy,accentedHitsPositionParter_accuracy,def_hits_accuracy,def_takedown_accuracy,def_accentedHits_accuracy,def_hitsHead_accuracy,def_hitsBody_accuracy,def_hitsLegs_accuracy,def_accentedHitsPositionDistance_accuracy,def_accentedHitsPositionClinch_accuracy,def_accentedHitsPositionParter_accuracy,DEC_percent,DQ_percent,KO_percent,SUB_percent,hits_PM,takedown_PM,accentedHits_PM,hitsHead_PM,hitsBody_PM,hitsLegs_PM,accentedHitsPositionDistance_PM,accentedHitsPositionClinch_PM,accentedHitsPositionParter_PM,def_hits_PM,def_takedown_PM,def_accentedHits_PM,def_hitsHead_PM,def_hitsBody_PM,def_hitsLegs_PM,def_accentedHitsPositionDistance_PM,def_accentedHitsPositionClinch_PM,def_accentedHitsPositionParter_PM,knockdowns_PM,def_knockdowns_PM,protectionPassage_PM,def_protectionPassage_PM,win_streak,loose_s

In [26]:
# Create prediction vector
X_df = pd.DataFrame(index=[0])

X_df = X_df.join(fighter1_stats[num_cols].add_prefix("f1_"))
X_df = X_df.join(fighter2_stats[num_cols].add_prefix("f2_"))

X_df.loc[0, ['f1_age', 'f2_age', 'f1_odds', 'f2_odds']] = f1_age, f2_age, f1_odd, f2_odd

X_df[['weightCategory.id', 'city', 'country', 'is_fight_night', 'timezone']] = \
    weightCategory_id, city, country, is_fight_night, time_zone
print(X_df.shape)
X_df

(1, 201)


,f1_cumsum_duration,f1_cumsum_winner,f1_cumsum_hitsTotal,f1_cumsum_hitsSuccessful,f1_cumsum_takedownTotal,f1_cumsum_takedownSuccessful,f1_cumsum_submissionAttempts,f1_cumsum_takeovers,f1_cumsum_accentedHitsTotal,f1_cumsum_accentedHitsSuccessful,f1_cumsum_knockdowns,f1_cumsum_protectionPassage,f1_cumsum_hitsHeadTotal,f1_cumsum_hitsHeadSuccessful,f1_cumsum_hitsBodyTotal,f1_cumsum_hitsBodySuccessful,f1_cumsum_hitsLegsTotal,f1_cumsum_hitsLegsSuccessful,f1_cumsum_accentedHitsPositionDistanceTotal,f1_cumsum_accentedHitsPositionDistanceSuccessful,f1_cumsum_accentedHitsPositionClinchTotal,f1_cumsum_accentedHitsPositionClinchSuccessful,f1_cumsum_accentedHitsPositionParterTotal,f1_cumsum_accentedHitsPositionParterSuccessful,f1_cumsum_def_hitsTotal,f1_cumsum_def_hitsSuccessful,f1_cumsum_def_takedownTotal,f1_cumsum_def_takedownSuccessful,f1_cumsum_def_submissionAttempts,f1_cumsum_def_takeovers,f1_cumsum_def_accentedHitsTotal,f1_cumsum_def_accentedHitsSuccessful,f1_cumsum_def_knockdowns,f1_cumsum_def_protectionPassage,f1_cumsum_def_hitsHeadTotal,f1_cumsum_def_hitsHeadSuccessful,f1_cumsum_def_hitsBodyTotal,f1_cumsum_def_hitsBodySuccessful,f1_cumsum_def_hitsLegsTotal,f1_cumsum_def_hitsLegsSuccessful,f1_cumsum_def_accentedHitsPositionDistanceTotal,f1_cumsum_def_accentedHitsPositionDistanceSuccessful,f1_cumsum_def_accentedHitsPositionClinchTotal,f1_cumsum_def_accentedHitsPositionClinchSuccessful,f1_cumsum_def_accentedHitsPositionParterTotal,f1_cumsum_def_accentedHitsPositionParterSuccessful,f1_cumsum_winMethods_[DEC],f1_cumsum_winMethods_[DQ],f1_cumsum_winMethods_[KO],f1_cumsum_winMethods_[SUB],f1_hits_accuracy,f1_takedown_accuracy,f1_accentedHits_accuracy,f1_hitsHead_accuracy,f1_hitsBody_accuracy,f1_hitsLegs_accuracy,f1_accentedHitsPositionDistance_accuracy,f1_accentedHitsPositionClinch_accuracy,f1_accentedHitsPositionParter_accuracy,f1_DEC_percent,f1_DQ_percent,f1_KO_percent,f1_SUB_percent,f1_hits_PM,f1_takedown_PM,f1_accentedHits_PM,f1_hitsHead_PM,f1_hitsBody_PM,f1_hitsLegs_PM,f1_accentedHitsPositionDistance_PM,f1_accentedHitsPositionClinch_PM,f1_accentedHitsPositionParter_PM,f1_knockdowns_PM,f1_protectionPassage_PM,f1_win_streak,f1_loose_streak,f1_def_hits_accuracy,f1_def_takedown_accuracy,f1_def_accentedHits_accuracy,f1_def_hitsHead_accuracy,f1_def_hitsBody_accuracy,f1_def_hitsLegs_accuracy,f1_def_accentedHitsPositionDistance_accuracy,f1_def_accentedHitsPositionClinch_accuracy,f1_def_accentedHitsPositionParter_accuracy,f1_def_hits_PM,f1_def_takedown_PM,f1_def_accentedHits_PM,f1_def_hitsHead_PM,f1_def_hitsBody_PM,f1_def_hitsLegs_PM,f1_def_accentedHitsPositionDistance_PM,f1_def_accentedHitsPositionClinch_PM,f1_def_accentedHitsPositionParter_PM,f1_def_knockdowns_PM,f1_def_protectionPassage_PM,f1_odds,f2_cumsum_duration,f2_cumsum_winner,f2_cumsum_hitsTotal,f2_cumsum_hitsSuccessful,f2_cumsum_takedownTotal,f2_cumsum_takedownSuccessful,f2_cumsum_submissionAttempts,f2_cumsum_takeovers,f2_cumsum_accentedHitsTotal,f2_cumsum_accentedHitsSuccessful,f2_cumsum_knockdowns,f2_cumsum_protectionPassage,f2_cumsum_hitsHeadTotal,f2_cumsum_hitsHeadSuccessful,f2_cumsum_hitsBodyTotal,f2_cumsum_hitsBodySuccessful,f2_cumsum_hitsLegsTotal,f2_cumsum_hitsLegsSuccessful,f2_cumsum_accentedHitsPositionDistanceTotal,f2_cumsum_accentedHitsPositionDistanceSuccessful,f2_cumsum_accentedHitsPositionClinchTotal,f2_cumsum_accentedHitsPositionClinchSuccessful,f2_cumsum_accentedHitsPositionParterTotal,f2_cumsum_accentedHitsPositionParterSuccessful,f2_cumsum_def_hitsTotal,f2_cumsum_def_hitsSuccessful,f2_cumsum_def_takedownTotal,f2_cumsum_def_takedownSuccessful,f2_cumsum_def_submissionAttempts,f2_cumsum_def_takeovers,f2_cumsum_def_accentedHitsTotal,f2_cumsum_def_accentedHitsSuccessful,f2_cumsum_def_knockdowns,f2_cumsum_def_protectionPassage,f2_cumsum_def_hitsHeadTotal,f2_cumsum_def_hitsHeadSuccessful,f2_cumsum_def_hitsBodyTotal,f2_cumsum_def_hitsBodySuccessful,f2_cumsum_def_hitsLegsTotal,f2_cumsum_def_hitsLegsSuccessful,f2_cumsum_def_accentedHitsPositionDistanceTotal,f2_cumsum_def_

In [27]:
X_df[f1_static_cols] = f1_static_stats
X_df[f2_static_cols] = f2_static_stats
print(X_df.shape)
X_df

(1, 215)


,f1_cumsum_duration,f1_cumsum_winner,f1_cumsum_hitsTotal,f1_cumsum_hitsSuccessful,f1_cumsum_takedownTotal,f1_cumsum_takedownSuccessful,f1_cumsum_submissionAttempts,f1_cumsum_takeovers,f1_cumsum_accentedHitsTotal,f1_cumsum_accentedHitsSuccessful,f1_cumsum_knockdowns,f1_cumsum_protectionPassage,f1_cumsum_hitsHeadTotal,f1_cumsum_hitsHeadSuccessful,f1_cumsum_hitsBodyTotal,f1_cumsum_hitsBodySuccessful,f1_cumsum_hitsLegsTotal,f1_cumsum_hitsLegsSuccessful,f1_cumsum_accentedHitsPositionDistanceTotal,f1_cumsum_accentedHitsPositionDistanceSuccessful,f1_cumsum_accentedHitsPositionClinchTotal,f1_cumsum_accentedHitsPositionClinchSuccessful,f1_cumsum_accentedHitsPositionParterTotal,f1_cumsum_accentedHitsPositionParterSuccessful,f1_cumsum_def_hitsTotal,f1_cumsum_def_hitsSuccessful,f1_cumsum_def_takedownTotal,f1_cumsum_def_takedownSuccessful,f1_cumsum_def_submissionAttempts,f1_cumsum_def_takeovers,f1_cumsum_def_accentedHitsTotal,f1_cumsum_def_accentedHitsSuccessful,f1_cumsum_def_knockdowns,f1_cumsum_def_protectionPassage,f1_cumsum_def_hitsHeadTotal,f1_cumsum_def_hitsHeadSuccessful,f1_cumsum_def_hitsBodyTotal,f1_cumsum_def_hitsBodySuccessful,f1_cumsum_def_hitsLegsTotal,f1_cumsum_def_hitsLegsSuccessful,f1_cumsum_def_accentedHitsPositionDistanceTotal,f1_cumsum_def_accentedHitsPositionDistanceSuccessful,f1_cumsum_def_accentedHitsPositionClinchTotal,f1_cumsum_def_accentedHitsPositionClinchSuccessful,f1_cumsum_def_accentedHitsPositionParterTotal,f1_cumsum_def_accentedHitsPositionParterSuccessful,f1_cumsum_winMethods_[DEC],f1_cumsum_winMethods_[DQ],f1_cumsum_winMethods_[KO],f1_cumsum_winMethods_[SUB],f1_hits_accuracy,f1_takedown_accuracy,f1_accentedHits_accuracy,f1_hitsHead_accuracy,f1_hitsBody_accuracy,f1_hitsLegs_accuracy,f1_accentedHitsPositionDistance_accuracy,f1_accentedHitsPositionClinch_accuracy,f1_accentedHitsPositionParter_accuracy,f1_DEC_percent,f1_DQ_percent,f1_KO_percent,f1_SUB_percent,f1_hits_PM,f1_takedown_PM,f1_accentedHits_PM,f1_hitsHead_PM,f1_hitsBody_PM,f1_hitsLegs_PM,f1_accentedHitsPositionDistance_PM,f1_accentedHitsPositionClinch_PM,f1_accentedHitsPositionParter_PM,f1_knockdowns_PM,f1_protectionPassage_PM,f1_win_streak,f1_loose_streak,f1_def_hits_accuracy,f1_def_takedown_accuracy,f1_def_accentedHits_accuracy,f1_def_hitsHead_accuracy,f1_def_hitsBody_accuracy,f1_def_hitsLegs_accuracy,f1_def_accentedHitsPositionDistance_accuracy,f1_def_accentedHitsPositionClinch_accuracy,f1_def_accentedHitsPositionParter_accuracy,f1_def_hits_PM,f1_def_takedown_PM,f1_def_accentedHits_PM,f1_def_hitsHead_PM,f1_def_hitsBody_PM,f1_def_hitsLegs_PM,f1_def_accentedHitsPositionDistance_PM,f1_def_accentedHitsPositionClinch_PM,f1_def_accentedHitsPositionParter_PM,f1_def_knockdowns_PM,f1_def_protectionPassage_PM,f1_odds,f2_cumsum_duration,f2_cumsum_winner,f2_cumsum_hitsTotal,f2_cumsum_hitsSuccessful,f2_cumsum_takedownTotal,f2_cumsum_takedownSuccessful,f2_cumsum_submissionAttempts,f2_cumsum_takeovers,f2_cumsum_accentedHitsTotal,f2_cumsum_accentedHitsSuccessful,f2_cumsum_knockdowns,f2_cumsum_protectionPassage,f2_cumsum_hitsHeadTotal,f2_cumsum_hitsHeadSuccessful,f2_cumsum_hitsBodyTotal,f2_cumsum_hitsBodySuccessful,f2_cumsum_hitsLegsTotal,f2_cumsum_hitsLegsSuccessful,f2_cumsum_accentedHitsPositionDistanceTotal,f2_cumsum_accentedHitsPositionDistanceSuccessful,f2_cumsum_accentedHitsPositionClinchTotal,f2_cumsum_accentedHitsPositionClinchSuccessful,f2_cumsum_accentedHitsPositionParterTotal,f2_cumsum_accentedHitsPositionParterSuccessful,f2_cumsum_def_hitsTotal,f2_cumsum_def_hitsSuccessful,f2_cumsum_def_takedownTotal,f2_cumsum_def_takedownSuccessful,f2_cumsum_def_submissionAttempts,f2_cumsum_def_takeovers,f2_cumsum_def_accentedHitsTotal,f2_cumsum_def_accentedHitsSuccessful,f2_cumsum_def_knockdowns,f2_cumsum_def_protectionPassage,f2_cumsum_def_hitsHeadTotal,f2_cumsum_def_hitsHeadSuccessful,f2_cumsum_def_hitsBodyTotal,f2_cumsum_def_hitsBodySuccessful,f2_cumsum_def_hitsLegsTotal,f2_cumsum_def_hitsLegsSuccessful,f2_cumsum_def_accentedHitsPositionDistanceTotal,f2_cumsum_def_

In [28]:
binary_fighter_cols = []
for prefix in ["f1_", "f2_"]:
    for key in ["isHomeCity", "isHomeCountry", "isHomeTimezone"]:
        binary_fighter_cols.append(prefix + key)

binary_stats = []
binary_cols = ['city', 'country', 'timezone']
for prefix in ["f1_", "f2_"]:
    for col in binary_cols:
        binary_stats.append(int(X_df.loc[0, prefix + col] == X_df.loc[0, col]))

X_df[binary_fighter_cols] = binary_stats
print(X_df.shape)
X_df[binary_fighter_cols]

(1, 221)


,f1_isHomeCity,f1_isHomeCountry,f1_isHomeTimezone,f2_isHomeCity,f2_isHomeCountry,f2_isHomeTimezone
0,0,0,1,0,0,0


In [29]:
X_df[f1_cat_cols + f2_cat_cols] = X_df[f1_cat_cols + f2_cat_cols].fillna('unknown')
X_df[f1_cat_cols + f2_cat_cols]

,f1_country,f1_city,f1_timezone,f2_country,f2_city,f2_timezone
0,Germany,unknown,Europe/Berlin,Azerbaijan,unknown,Asia/Baku


In [30]:
# Difference
diff_cols = []
fighter1_stat_cols = generated_features['fighter1_stats']
for col in fighter1_stat_cols:
    new_col_name = col[3:] + '_difference'
    diff_cols.append(new_col_name)
    X_df[new_col_name] = X_df['f1_' + col[3:]].astype(float) - X_df['f2_' + col[3:]].astype(float)
X_df[diff_cols]

,cumsum_duration_difference,cumsum_winner_difference,cumsum_hitsTotal_difference,cumsum_hitsSuccessful_difference,cumsum_takedownTotal_difference,cumsum_takedownSuccessful_difference,cumsum_submissionAttempts_difference,cumsum_takeovers_difference,cumsum_accentedHitsTotal_difference,cumsum_accentedHitsSuccessful_difference,cumsum_knockdowns_difference,cumsum_protectionPassage_difference,cumsum_hitsHeadTotal_difference,cumsum_hitsHeadSuccessful_difference,cumsum_hitsBodyTotal_difference,cumsum_hitsBodySuccessful_difference,cumsum_hitsLegsTotal_difference,cumsum_hitsLegsSuccessful_difference,cumsum_accentedHitsPositionDistanceTotal_difference,cumsum_accentedHitsPositionDistanceSuccessful_difference,cumsum_accentedHitsPositionClinchTotal_difference,cumsum_accentedHitsPositionClinchSuccessful_difference,cumsum_accentedHitsPositionParterTotal_difference,cumsum_accentedHitsPositionParterSuccessful_difference,cumsum_def_hitsTotal_difference,cumsum_def_hitsSuccessful_difference,cumsum_def_takedownTotal_difference,cumsum_def_takedownSuccessful_difference,cumsum_def_submissionAttempts_difference,cumsum_def_takeovers_difference,cumsum_def_accentedHitsTotal_difference,cumsum_def_accentedHitsSuccessful_difference,cumsum_def_knockdowns_difference,cumsum_def_protectionPassage_difference,cumsum_def_hitsHeadTotal_difference,cumsum_def_hitsHeadSuccessful_difference,cumsum_def_hitsBodyTotal_difference,cumsum_def_hitsBodySuccessful_difference,cumsum_def_hitsLegsTotal_difference,cumsum_def_hitsLegsSuccessful_difference,cumsum_def_accentedHitsPositionDistanceTotal_difference,cumsum_def_accentedHitsPositionDistanceSuccessful_difference,cumsum_def_accentedHitsPositionClinchTotal_difference,cumsum_def_accentedHitsPositionClinchSuccessful_difference,cumsum_def_accentedHitsPositionParterTotal_difference,cumsum_def_accentedHitsPositionParterSuccessful_difference,cumsum_winMethods_[DEC]_difference,cumsum_winMethods_[DQ]_difference,cumsum_winMethods_[KO]_difference,cumsum_winMethods_[SUB]_difference,hits_accuracy_difference,takedown_accuracy_difference,accentedHits_accuracy_difference,hitsHead_accuracy_difference,hitsBody_accuracy_difference,hitsLegs_accuracy_difference,accentedHitsPositionDistance_accuracy_difference,accentedHitsPositionClinch_accuracy_difference,accentedHitsPositionParter_accuracy_difference,DEC_percent_difference,DQ_percent_difference,KO_percent_difference,SUB_percent_difference,hits_PM_difference,takedown_PM_difference,accentedHits_PM_difference,hitsHead_PM_difference,hitsBody_PM_difference,hitsLegs_PM_difference,accentedHitsPositionDistance_PM_difference,accentedHitsPositionClinch_PM_difference,accentedHitsPositionParter_PM_difference,knockdowns_PM_difference,protectionPassage_PM_difference,win_streak_difference,loose_streak_difference,def_hits_accuracy_difference,def_takedown_accuracy_difference,def_accentedHits_accuracy_difference,def_hitsHead_accuracy_difference,def_hitsBody_accuracy_difference,def_hitsLegs_accuracy_difference,def_accentedHitsPositionDistance_accuracy_difference,def_accentedHitsPositionClinch_accuracy_difference,def_accentedHitsPositionParter_accuracy_difference,def_hits_PM_difference,def_takedown_PM_difference,def_accentedHits_PM_difference,def_hitsHead_PM_difference,def_hitsBody_PM_difference,def_hitsLegs_PM_difference,def_accentedHitsPositionDistance_PM_difference,def_accentedHitsPositionClinch_PM_difference,def_accentedHitsPositionParter_PM_difference,def_knockdowns_PM_difference,def_protectionPassage_PM_difference,odds_difference,age_difference,height_difference,weight_difference,armSpan_difference,legSwing_difference,isHomeCity_difference,isHomeCountry_difference,isHomeTimezone_difference
0,-783.0,-3.0,-71.0,-37.0,-12.0,-2.0,4.0,-5.0,-56.0,-27.0,0.0,0.0,-48.0,-18.0,-7.0,-8.0,-1.0,-1.0,-32.0,-8.0,-21.0,-17.0,-3.0,-2.0,-44.0,18.0,-2.0,0.0,0.0,3.0,-53.0,15.0,0.0,0.0,-40.0,19.0,-12.0,-5.0,-1.0,1.0,-61.0,7.0,-13.0,-8.0,21.0,16.0,-4.0,0.0,1.0,1.0,0.020292,-0.133333,-0.031499,-0.011154,-0.19697,0.0,0.047059,-0.22413

In [31]:
X_df_reversed = X_df.copy()

reversed_cols = []
for col in X_df.columns:
    if 'f2' in col:
        new_col_name = col.replace('f2', 'f1')
    elif 'f1' in col:
        new_col_name = col.replace('f1', 'f2')
    else:
        new_col_name = col
    reversed_cols.append(new_col_name)
X_df_reversed.columns = reversed_cols
X_df_reversed

,f2_cumsum_duration,f2_cumsum_winner,f2_cumsum_hitsTotal,f2_cumsum_hitsSuccessful,f2_cumsum_takedownTotal,f2_cumsum_takedownSuccessful,f2_cumsum_submissionAttempts,f2_cumsum_takeovers,f2_cumsum_accentedHitsTotal,f2_cumsum_accentedHitsSuccessful,f2_cumsum_knockdowns,f2_cumsum_protectionPassage,f2_cumsum_hitsHeadTotal,f2_cumsum_hitsHeadSuccessful,f2_cumsum_hitsBodyTotal,f2_cumsum_hitsBodySuccessful,f2_cumsum_hitsLegsTotal,f2_cumsum_hitsLegsSuccessful,f2_cumsum_accentedHitsPositionDistanceTotal,f2_cumsum_accentedHitsPositionDistanceSuccessful,f2_cumsum_accentedHitsPositionClinchTotal,f2_cumsum_accentedHitsPositionClinchSuccessful,f2_cumsum_accentedHitsPositionParterTotal,f2_cumsum_accentedHitsPositionParterSuccessful,f2_cumsum_def_hitsTotal,f2_cumsum_def_hitsSuccessful,f2_cumsum_def_takedownTotal,f2_cumsum_def_takedownSuccessful,f2_cumsum_def_submissionAttempts,f2_cumsum_def_takeovers,f2_cumsum_def_accentedHitsTotal,f2_cumsum_def_accentedHitsSuccessful,f2_cumsum_def_knockdowns,f2_cumsum_def_protectionPassage,f2_cumsum_def_hitsHeadTotal,f2_cumsum_def_hitsHeadSuccessful,f2_cumsum_def_hitsBodyTotal,f2_cumsum_def_hitsBodySuccessful,f2_cumsum_def_hitsLegsTotal,f2_cumsum_def_hitsLegsSuccessful,f2_cumsum_def_accentedHitsPositionDistanceTotal,f2_cumsum_def_accentedHitsPositionDistanceSuccessful,f2_cumsum_def_accentedHitsPositionClinchTotal,f2_cumsum_def_accentedHitsPositionClinchSuccessful,f2_cumsum_def_accentedHitsPositionParterTotal,f2_cumsum_def_accentedHitsPositionParterSuccessful,f2_cumsum_winMethods_[DEC],f2_cumsum_winMethods_[DQ],f2_cumsum_winMethods_[KO],f2_cumsum_winMethods_[SUB],f2_hits_accuracy,f2_takedown_accuracy,f2_accentedHits_accuracy,f2_hitsHead_accuracy,f2_hitsBody_accuracy,f2_hitsLegs_accuracy,f2_accentedHitsPositionDistance_accuracy,f2_accentedHitsPositionClinch_accuracy,f2_accentedHitsPositionParter_accuracy,f2_DEC_percent,f2_DQ_percent,f2_KO_percent,f2_SUB_percent,f2_hits_PM,f2_takedown_PM,f2_accentedHits_PM,f2_hitsHead_PM,f2_hitsBody_PM,f2_hitsLegs_PM,f2_accentedHitsPositionDistance_PM,f2_accentedHitsPositionClinch_PM,f2_accentedHitsPositionParter_PM,f2_knockdowns_PM,f2_protectionPassage_PM,f2_win_streak,f2_loose_streak,f2_def_hits_accuracy,f2_def_takedown_accuracy,f2_def_accentedHits_accuracy,f2_def_hitsHead_accuracy,f2_def_hitsBody_accuracy,f2_def_hitsLegs_accuracy,f2_def_accentedHitsPositionDistance_accuracy,f2_def_accentedHitsPositionClinch_accuracy,f2_def_accentedHitsPositionParter_accuracy,f2_def_hits_PM,f2_def_takedown_PM,f2_def_accentedHits_PM,f2_def_hitsHead_PM,f2_def_hitsBody_PM,f2_def_hitsLegs_PM,f2_def_accentedHitsPositionDistance_PM,f2_def_accentedHitsPositionClinch_PM,f2_def_accentedHitsPositionParter_PM,f2_def_knockdowns_PM,f2_def_protectionPassage_PM,f2_odds,f1_cumsum_duration,f1_cumsum_winner,f1_cumsum_hitsTotal,f1_cumsum_hitsSuccessful,f1_cumsum_takedownTotal,f1_cumsum_takedownSuccessful,f1_cumsum_submissionAttempts,f1_cumsum_takeovers,f1_cumsum_accentedHitsTotal,f1_cumsum_accentedHitsSuccessful,f1_cumsum_knockdowns,f1_cumsum_protectionPassage,f1_cumsum_hitsHeadTotal,f1_cumsum_hitsHeadSuccessful,f1_cumsum_hitsBodyTotal,f1_cumsum_hitsBodySuccessful,f1_cumsum_hitsLegsTotal,f1_cumsum_hitsLegsSuccessful,f1_cumsum_accentedHitsPositionDistanceTotal,f1_cumsum_accentedHitsPositionDistanceSuccessful,f1_cumsum_accentedHitsPositionClinchTotal,f1_cumsum_accentedHitsPositionClinchSuccessful,f1_cumsum_accentedHitsPositionParterTotal,f1_cumsum_accentedHitsPositionParterSuccessful,f1_cumsum_def_hitsTotal,f1_cumsum_def_hitsSuccessful,f1_cumsum_def_takedownTotal,f1_cumsum_def_takedownSuccessful,f1_cumsum_def_submissionAttempts,f1_cumsum_def_takeovers,f1_cumsum_def_accentedHitsTotal,f1_cumsum_def_accentedHitsSuccessful,f1_cumsum_def_knockdowns,f1_cumsum_def_protectionPassage,f1_cumsum_def_hitsHeadTotal,f1_cumsum_def_hitsHeadSuccessful,f1_cumsum_def_hitsBodyTotal,f1_cumsum_def_hitsBodySuccessful,f1_cumsum_def_hitsLegsTotal,f1_cumsum_def_hitsLegsSuccessful,f1_cumsum_def_accentedHitsPositionDistanceTotal,f1_cumsum_def_

In [32]:
diff_cols = []
for col in fighter1_stat_cols:
    new_col_name = col[3:] + '_difference'
    diff_cols.append(new_col_name)
    X_df_reversed[new_col_name] = X_df_reversed['f1_' + col[3:]].astype(float) - \
        X_df_reversed['f2_' + col[3:]].astype(float)
X_df_reversed[diff_cols]

,cumsum_duration_difference,cumsum_winner_difference,cumsum_hitsTotal_difference,cumsum_hitsSuccessful_difference,cumsum_takedownTotal_difference,cumsum_takedownSuccessful_difference,cumsum_submissionAttempts_difference,cumsum_takeovers_difference,cumsum_accentedHitsTotal_difference,cumsum_accentedHitsSuccessful_difference,cumsum_knockdowns_difference,cumsum_protectionPassage_difference,cumsum_hitsHeadTotal_difference,cumsum_hitsHeadSuccessful_difference,cumsum_hitsBodyTotal_difference,cumsum_hitsBodySuccessful_difference,cumsum_hitsLegsTotal_difference,cumsum_hitsLegsSuccessful_difference,cumsum_accentedHitsPositionDistanceTotal_difference,cumsum_accentedHitsPositionDistanceSuccessful_difference,cumsum_accentedHitsPositionClinchTotal_difference,cumsum_accentedHitsPositionClinchSuccessful_difference,cumsum_accentedHitsPositionParterTotal_difference,cumsum_accentedHitsPositionParterSuccessful_difference,cumsum_def_hitsTotal_difference,cumsum_def_hitsSuccessful_difference,cumsum_def_takedownTotal_difference,cumsum_def_takedownSuccessful_difference,cumsum_def_submissionAttempts_difference,cumsum_def_takeovers_difference,cumsum_def_accentedHitsTotal_difference,cumsum_def_accentedHitsSuccessful_difference,cumsum_def_knockdowns_difference,cumsum_def_protectionPassage_difference,cumsum_def_hitsHeadTotal_difference,cumsum_def_hitsHeadSuccessful_difference,cumsum_def_hitsBodyTotal_difference,cumsum_def_hitsBodySuccessful_difference,cumsum_def_hitsLegsTotal_difference,cumsum_def_hitsLegsSuccessful_difference,cumsum_def_accentedHitsPositionDistanceTotal_difference,cumsum_def_accentedHitsPositionDistanceSuccessful_difference,cumsum_def_accentedHitsPositionClinchTotal_difference,cumsum_def_accentedHitsPositionClinchSuccessful_difference,cumsum_def_accentedHitsPositionParterTotal_difference,cumsum_def_accentedHitsPositionParterSuccessful_difference,cumsum_winMethods_[DEC]_difference,cumsum_winMethods_[DQ]_difference,cumsum_winMethods_[KO]_difference,cumsum_winMethods_[SUB]_difference,hits_accuracy_difference,takedown_accuracy_difference,accentedHits_accuracy_difference,hitsHead_accuracy_difference,hitsBody_accuracy_difference,hitsLegs_accuracy_difference,accentedHitsPositionDistance_accuracy_difference,accentedHitsPositionClinch_accuracy_difference,accentedHitsPositionParter_accuracy_difference,DEC_percent_difference,DQ_percent_difference,KO_percent_difference,SUB_percent_difference,hits_PM_difference,takedown_PM_difference,accentedHits_PM_difference,hitsHead_PM_difference,hitsBody_PM_difference,hitsLegs_PM_difference,accentedHitsPositionDistance_PM_difference,accentedHitsPositionClinch_PM_difference,accentedHitsPositionParter_PM_difference,knockdowns_PM_difference,protectionPassage_PM_difference,win_streak_difference,loose_streak_difference,def_hits_accuracy_difference,def_takedown_accuracy_difference,def_accentedHits_accuracy_difference,def_hitsHead_accuracy_difference,def_hitsBody_accuracy_difference,def_hitsLegs_accuracy_difference,def_accentedHitsPositionDistance_accuracy_difference,def_accentedHitsPositionClinch_accuracy_difference,def_accentedHitsPositionParter_accuracy_difference,def_hits_PM_difference,def_takedown_PM_difference,def_accentedHits_PM_difference,def_hitsHead_PM_difference,def_hitsBody_PM_difference,def_hitsLegs_PM_difference,def_accentedHitsPositionDistance_PM_difference,def_accentedHitsPositionClinch_PM_difference,def_accentedHitsPositionParter_PM_difference,def_knockdowns_PM_difference,def_protectionPassage_PM_difference,odds_difference,age_difference,height_difference,weight_difference,armSpan_difference,legSwing_difference,isHomeCity_difference,isHomeCountry_difference,isHomeTimezone_difference
0,783.0,3.0,71.0,37.0,12.0,2.0,-4.0,5.0,56.0,27.0,0.0,0.0,48.0,18.0,7.0,8.0,1.0,1.0,32.0,8.0,21.0,17.0,3.0,2.0,44.0,-18.0,2.0,0.0,0.0,-3.0,53.0,-15.0,0.0,0.0,40.0,-19.0,12.0,5.0,1.0,-1.0,61.0,-7.0,13.0,8.0,-21.0,-16.0,4.0,0.0,-1.0,-1.0,-0.020292,0.133333,0.031499,0.011154,0.19697,0.0,-0.047059,0.224138,NaN,0.25,0.0,-1.0,-1.0

In [38]:
X_df[['f1_cumsum_hitsTotal', 'f2_cumsum_hitsTotal']]

,f1_cumsum_hitsTotal,f2_cumsum_hitsTotal
0,105,176


In [34]:
X_df[clf.feature_names_]

,age_difference,armSpan_difference,cumsum_def_hitsTotal_difference,cumsum_hitsSuccessful_difference,cumsum_submissionAttempts_difference,def_accentedHitsPositionDistance_accuracy_difference,def_hitsHead_accuracy_difference,def_hitsLegs_PM_difference,def_hitsLegs_accuracy_difference,cumsum_takeovers_difference,cumsum_takedownTotal_difference,isHomeCity_difference,isHomeCountry_difference,takedown_PM_difference,win_streak_difference
0,-3.0,-2.54,-44.0,-37.0,4.0,0.196064,0.243861,0.607866,0.090643,-5.0,-12.0,0.0,0.0,-0.08,0.0


In [35]:
X_df_reversed[clf.feature_names_]

,age_difference,armSpan_difference,cumsum_def_hitsTotal_difference,cumsum_hitsSuccessful_difference,cumsum_submissionAttempts_difference,def_accentedHitsPositionDistance_accuracy_difference,def_hitsHead_accuracy_difference,def_hitsLegs_PM_difference,def_hitsLegs_accuracy_difference,cumsum_takeovers_difference,cumsum_takedownTotal_difference,isHomeCity_difference,isHomeCountry_difference,takedown_PM_difference,win_streak_difference
0,3.0,2.54,44.0,37.0,-4.0,-0.196064,-0.243861,-0.607866,-0.090643,5.0,12.0,0.0,0.0,0.08,0.0


In [39]:
y_proba1 = clf.predict_proba(X_df[clf.feature_names_])[:, 1]
y_proba2 = clf.predict_proba(X_df_reversed[clf.feature_names_])[:, 0]
y_proba = (y_proba1 + y_proba2) / 2
y_proba1, y_proba2, y_proba

(array([0.43673265]), array([0.44452294]), array([0.44062779]))